In [41]:
import import_ipynb
from LoadPicture import *

import numpy as np
import glob
import matplotlib.pyplot as plt

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [42]:
batch_size = 32
class_names = ['FaceOnly', 'NoFace']

## Model callbacks

In [43]:
callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath='../log/cps/{model.name}_cp_{epoch}', 
                        verbose=1, 
                        save_weights_only=False,
                        save_freq=5*batch_size) # Checkpoint is saved every five epochs
]

## Data parameters

In [44]:
image_height = 224
image_width = 224

## Creating the training dataset

In [45]:
face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    labels='inferred',
    label_mode='binary',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

#face_training_ds.map(lambda x, y: (tf.image.resize(x, (image_height, image_width)), y))
face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 20 files belonging to 2 classes.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [46]:
validation_ds = []

## Creating the test dataset

In [47]:
test_ds = []

## Load the base model

In [48]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50V2(include_top=False)
base_model.trainable = False

## Creating a new model

In [49]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

## Training the new model with new data

In [50]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(
            face_training_ds, 
            epochs=20, 
            callbacks=callbacks, 
            #validation_data=...
        )

Epoch 1/20


ValueError: in user code:

    File "c:\Users\Zolkin\anaconda3\envs\dl_faceRecognition\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Zolkin\anaconda3\envs\dl_faceRecognition\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Zolkin\anaconda3\envs\dl_faceRecognition\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Zolkin\anaconda3\envs\dl_faceRecognition\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Zolkin\anaconda3\envs\dl_faceRecognition\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Zolkin\anaconda3\envs\dl_faceReco

## View Results

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/nonFace_10.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

img_array = preprocess_input(x)

preds = model.predict(img_array)
score = tf.nn.softmax(preds)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', tf.keras.applications.imagenet_utils.decode_predictions(preds)[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 238ms/step
This image most likely belongs to FaceOnly with a 100.00 percent confidence.
